In [1]:
%matplotlib inline

import json
import pandas as pd
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
import numpy as np

In [2]:
with open('2017forbes2000.json', 'r') as f:
    data = json.load(f)

In [3]:
df = json_normalize(data)
print list(df)
imp = set()

[u'assets', u'ceo', u'country', u'headquarters', u'imageUri', u'industry', u'marketValue', u'name', u'position', u'profits', u'rank', u'revenue', u'squareImage', u'state', u'thumbnail', u'uri']


In [ ]:
imp = set(df.sort_values(by='revenue', ascending=False).head(n=50).name)

In [4]:
imp = imp.union(set(df.sort_values(by='profits', ascending=False).head(n=50).name))

In [ ]:
imp = imp.union(set(df.sort_values(by='assets', ascending=False).head(n=50).name))

In [ ]:
imp = imp.union(set(df.sort_values(by='marketValue', ascending=False).head(n=100).name))

In [ ]:
len(imp)

In [5]:
shortlist = df[df.name.isin(imp)]

In [6]:
shortlist.industry.unique()

array([u'Telecommunications services', u'Regional Banks',
       u'Pharmaceuticals', u'Computer Services', u'Tobacco',
       u'Computer Hardware', u'Major Banks', u'Investment Services',
       u'Communications Equipment', u'Broadcasting & Cable',
       u'Auto & Truck Manufacturers', u'Oil & Gas Operations',
       u'Conglomerates', u'Biotechs', u'Home Improvement Retail',
       u'Semiconductors', u'Medical Equipment & Supplies',
       u'Software & Programming', u'Food Processing',
       u'Diversified Insurance', u'Household/Personal Care',
       u'Discount Stores'], dtype=object)

In [ ]:
shortlist[shortlist.industry=='Healthcare Services']

In [7]:
shortlist.industry = shortlist.industry.map({'Conglomerates': 'Conglomerates', 'Major Banks': 'Finance & Insurance',\
                                            'Telecommunications services': 'Telecommunications', 'Diversified Insurance': 'Finance & Insurance',\
                                            'Pharmaceuticals': 'Pharmaceuticals', 'Regional Banks': 'Finance & Insurance',\
                                            'Business & Personal Services': 'Technology', 'Computer Services': 'Technology',\
                                            'Tobacco':'Tobacco', 'Internet & Catalog Retail': 'Technology', 'Biotechs':'Pharmaceuticals',\
                                             'Beverages':'Beverages', 'Managed Health Care': 'Healthcare', 'Diversified Insurance': 'Finance & Insurance',\
                                             'Computer Hardware': 'Technology', 'Auto & Truck Manufacturers': 'Automotive',\
                                             'Oil & Gas Operations': 'Oil & Gas','Investment Services': 'Finance & Insurance',\
                                             'Drug Retail': 'Retail', 'Construction Services':'Construction',\
                                             'Communications Equipment': 'Technology', 'Iron & Steel': 'Conglomerates',\
                                             'Broadcasting & Cable': 'Telecommunications','Discount Stores': 'Retail',\
                                             'Electric Utilities': 'Electric Utilities', 'Healthcare Services': 'Healthcare',\
                                             'Diversified Metals & Mining':'Mining', 'Home Improvement Retail': 'Retail',\
                                             'Electronics': 'Technology', 'Semiconductors':'Technology',\
                                             'Life & Health Insurance': 'Finance & Insurance', 'Medical Equipment & Supplies': 'Consumer Goods',\
                                             'Food Retail': 'Retail', 'Software & Programming': 'Technology', 'Food Processing': 'Consumer Goods',\
                                             'Household/Personal Care': 'Consumer Goods','Consumer Financial Services': 'Finance & Insurance'\
                                            })

/Users/muhammaddawood/anaconda/lib/python2.7/site-packages/pandas/core/generic.py:2387: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [8]:
shortlist.loc[shortlist.name == 'Walt Disney', 'industry'] = 'Entertainment'

/Users/muhammaddawood/anaconda/lib/python2.7/site-packages/pandas/core/indexing.py:426: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [9]:
shortlist['length'] = shortlist.name.str.len()

/Users/muhammaddawood/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [10]:
shortlist.revenue = shortlist.revenue/1000
shortlist.marketValue = shortlist.marketValue/1000
shortlist.profits = shortlist.profits/1000
shortlist.assets= shortlist.assets/1000

In [ ]:
shortlist[shortlist.industry=='Technology']

In [ ]:
shortlist.industry.unique()

In [ ]:
jsonFile = {"name": "top50","children": []}
for entry in shortlist.sort_values(by='revenue', ascending=False).iterrows():
    if len(jsonFile['children'])==0:
        jsonFile['children'].append({"name":entry[1]['industry'], "children":[{'name':entry[1]['name'], 'size':entry[1]['profits'],'profit':entry[1]['profits'],'country':entry[1]['country'],'assets':entry[1]['assets'],"marketValue":entry[1]['marketValue']}]})
    else:
        entered = False
        for index,cat in enumerate(jsonFile['children']):
            if cat['name']==entry[1]['industry']:
                jsonFile['children'][index]['children'].append({'name':entry[1]['name'], 'size':entry[1]['profits'],'profit':entry[1]['profits'],'country':entry[1]['country'],'assets':entry[1]['assets'],"marketValue":entry[1]['marketValue']})
                entered = True
                break
        if not entered:
            jsonFile['children'].append({"name":entry[1]['industry'], "children":[{'name':entry[1]['name'], 'size':entry[1]['profits'],'profit':entry[1]['profits'],'country':entry[1]['country'],'assets':entry[1]['assets'],"marketValue":entry[1]['marketValue']}]})

In [ ]:
json.dumps(jsonFile)

In [ ]:
sums = []
names = []
for i,ind in enumerate(jsonFile["children"]):
    names.append(ind["name"])
    sums.append(0)
    for comp in ind["children"]:
        sums[i] += comp["size"]
print(names)
print(sums)

In [11]:
jsonFile = []
for entry in shortlist.sort_values(by='revenue', ascending=False).head(n=50).iterrows():
    jsonFile.append({"name":entry[1]["name"],"revenue":entry[1]["revenue"],"profit":entry[1]["profits"],"group":entry[1]["industry"]})
json.dumps(jsonFile)

'[{"profit": 13.643, "group": "Retail", "name": "Wal-Mart Stores", "revenue": 485.252}, {"profit": 17.1115, "group": "Automotive", "name": "Toyota Motor", "revenue": 249.902}, {"profit": 24.074, "group": "Finance & Insurance", "name": "Berkshire Hathaway", "revenue": 222.935}, {"profit": 45.217, "group": "Technology", "name": "Apple", "revenue": 217.481}, {"profit": 7.84, "group": "Oil & Gas", "name": "ExxonMobil", "revenue": 197.518}, {"profit": 19.320700000000002, "group": "Technology", "name": "Samsung Electronics", "revenue": 173.995}, {"profit": 9.4314, "group": "Automotive", "name": "Daimler", "revenue": 169.537}, {"profit": 9.427, "group": "Automotive", "name": "General Motors", "revenue": 166.38}, {"profit": 12.976, "group": "Telecommunications", "name": "AT&T", "revenue": 163.786}, {"profit": 41.9837, "group": "Finance & Insurance", "name": "ICBC", "revenue": 151.381}, {"profit": 34.9814, "group": "Finance & Insurance", "name": "China Construction Bank", "revenue": 134.242}, {

In [ ]:
jsonFile = {"categories": [],"series": ["Revenue","Profit"],"colors": ["#3498db","#e74c3c","#2ecc71"],"layers": [[],[]]}
for entry in shortlist.sort_values(by='revenue', ascending=False).head(n=50).iterrows():
    jsonFile["categories"].append(entry[1]["name"])
    jsonFile["layers"][0].append({"name":entry[1]["name"],"y":entry[1]["revenue"],"group":entry[1]["industry"]})
    jsonFile["layers"][1].append({"name":entry[1]["name"],"y":entry[1]["profits"],"group":entry[1]["industry"]})
json.dumps(jsonFile)

In [ ]:
plt.plot(shortlist.revenue, shortlist.length, "o")
plt.title("Gaussian Histogram")
plt.xlabel("Rev")
plt.ylabel("name")

In [ ]:
plt.show()